In [ ]:
import matplotlib.pyplot as plt
import rioxarray

from ocr import catalog, load_structures_destroyed
from ocr.utils import convert_coords, interpolate_to_30, lon_to_180, subset_region_xy
from ocr.wind import (
    apply_mode_calc,
    apply_wind_directional_convolution,
    classify_wind_directions,
    create_composite_bp_map,
    create_finescale_wind_direction,
)

In [ ]:
# Load in 270m risk

In [ ]:
riley = {}
riley['2011'] = catalog.get_dataset('2011-climate-run', version='v1').to_xarray(is_icechunk=True)
riley['2047'] = catalog.get_dataset('2047-climate-run', version='v1').to_xarray(is_icechunk=True)

In [ ]:
structures = load_structures_destroyed(fire_name='Eaton', target_crs='EPSG:5070')

In [ ]:
buffer = 20000
x_min, x_max = structures.geometry.x.min() - buffer, structures.geometry.x.max() + buffer
y_min, y_max = structures.geometry.y.min() - buffer, structures.geometry.y.max() + buffer

In [ ]:
subset = subset_region_xy(riley['2011'], [x_min, x_max], [y_min, y_max])

In [ ]:
[(lon_min, lat_max), (lon_max, lat_min)] = convert_coords(
    [(x_min, y_max), (x_max, y_min)], from_crs='EPSG:5070', to_crs='EPSG:4326'
)

In [ ]:
# expand the lats a smidge to make sure lat and lon dim lengths are both >1. it fails if dim length is 1. fixing this corner-case is low-priority.
lat_max += 0.2
lat_min -= 0.2
lon_max += 0.15
lon_min -= 0.15

In [ ]:
lat_min, lat_max

In [ ]:
lon_min, lon_max

In [ ]:
# load in the 30m product as a template
# Note: This does not have dask chunks
rps_30 = catalog.get_dataset('USFS-wildfire-risk-communities').to_xarray(is_icechunk=True)

In [ ]:
rps_30 = subset_region_xy(rps_30, [x_min, x_max], [y_min, y_max])

In [ ]:
rps_30

In [ ]:
subset_30 = interpolate_to_30(subset, rps_30)

In [ ]:
subset_30 = subset_30.rio.write_crs('EPSG:5070')

In [ ]:
subset_30.BP.plot()

In [ ]:
# reprojecting introduces a lot of nans which presents two TODOs
# (1) the convolution doesn't handle nans (at least presently). fill those with -9999s for now.
# TODO: this is an imperfect solution we want to fix to limit errors down the road. the
# (2) the reprojection decreases the useable data, meaning that we need to increase the buffer
# that we clip around the edges of the data tile
subset_30 = subset_30.rio.reproject('EPSG:4326').fillna(-9999)

In [ ]:
# Create a BP map smeared according to each of the 8 cardinal and intercardinal directions

In [ ]:
blurred_bp = apply_wind_directional_convolution(subset_30['BP'], iterations=3)

In [ ]:
# Load in the dataset of wind for fire weather days

In [ ]:
important_days = catalog.get_dataset('era5-fire-weather-days', version='v2').to_xarray(
    is_icechunk=False, xarray_open_kwargs={'engine': 'zarr'}
)

In [ ]:
important_days = lon_to_180(important_days)

In [ ]:
subset_wind = important_days.sel(
    latitude=slice(lat_max, lat_min), longitude=slice(lon_min, lon_max)
)
wind_directions = subset_wind['sfcWindfromdir']

In [ ]:
# we only have to do the wind direction classification once - want to get the distribution of each direction. prep this and store it like `fire_weather_days_v2.zarr`

In [ ]:
direction_indices = classify_wind_directions(wind_directions).chunk(dict(time=-1))

In [ ]:
direction_modes = apply_mode_calc(direction_indices).compute()

In [ ]:
direction_modes_reprojected = create_finescale_wind_direction(blurred_bp, direction_modes)

In [ ]:
wind_informed_bp = create_composite_bp_map(blurred_bp, direction_modes_reprojected)

In [ ]:
# load in the crps because we'll use it as the template to register the BP to

In [ ]:
crps = rioxarray.open_rasterio('s3://carbonplan-risks/wildfirecommunities/CRPS_CA.tif')

In [ ]:
subset_crps = crps.sel(x=slice(x_min, x_max), y=slice(y_max, y_min))

In [ ]:
wind_informed_bp = wind_informed_bp.rio.reproject_match(subset_crps)

In [ ]:
# TODO: decide how big of a buffer to trim it down. the size of the kernel plus a little to play it safe?

In [ ]:
wind_informed_bp = wind_informed_bp.sel(x=slice(x_min, x_max), y=slice(y_max, y_min))

In [ ]:
wind_informed_bp.plot()

In [ ]:
wind_informed_bp.to_dataset(name='BP').to_zarr(
    's3://carbonplan-risks/ocr/v0/intermediates/bp.zarr', mode='w'
)

In [ ]:
fig, ax = plt.subplots()
(wind_informed_bp * subset_crps).plot(ax=ax, vmin=0)
structures.plot(ax=ax)

In [ ]:
bp_orig = rioxarray.open_rasterio('s3://carbonplan-risks/wildfirecommunities/BP_CA.tif')

In [ ]:
fig, ax = plt.subplots()
(wind_informed_bp['BP'] * subset_crps).plot(vmin=0)
structures.plot(ax=ax)

In [ ]:
risk = (wind_informed_bp * subset_crps).to_dataset(name='risk')

In [ ]:
risk.where(risk > 0, 0).to_zarr('s3://carbonplan-risks/ocr/v0/intermediates/risk.zarr', mode='w')